In [1]:
import os
import pandas as pd
from oct_converter.readers import E2E
import matplotlib.pyplot as plt

In [2]:
# Load the CSV file
csv_file_path = 'AMD_Artificial_intelligence.csv'  # Replace with your actual file path
df = pd.read_csv(csv_file_path, header=1)

In [3]:
# Function to get acquisition date from .e2e files
def get_e2e_acquisition_date(file_path):
    try:
        fds = E2E(file_path)
        oct_volume = fds.read_oct_volume()
        tmp = oct_volume.pop()
        acquisition_date = pd.to_datetime(tmp.acquisition_date)
        return acquisition_date
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
        return None

In [4]:
# Root directory containing patient folders
root_directory = 'data'  # Replace with your actual root directory

# Create a dictionary to map acquisition dates to filenames
acquisition_date_to_info = {}

# Traverse through each patient directory and process .e2e files
for patient_folder in os.listdir(root_directory):
    patient_folder_path = os.path.join(root_directory, patient_folder)
    
    if os.path.isdir(patient_folder_path):
        for filename in os.listdir(patient_folder_path):
            if filename.endswith('.E2E'):
                file_path = os.path.join(patient_folder_path, filename)
                acquisition_date = get_e2e_acquisition_date(file_path)
                if acquisition_date:
                    # Store filename along with acquisition date
                    acquisition_date_to_info[acquisition_date] = {
                        'patient_folder': patient_folder,
                        'filename': filename
                    }


In [5]:
# Convert 'Scan Date' in CSV to datetime and match with acquisition dates
df['Scan Date'] = pd.to_datetime(df['Scan Date'], errors='coerce')

# Filter relevant rows and print them
relevant_rows = df[df['Scan Date'].isin(acquisition_date_to_info.keys())]

In [6]:
# Print whole rows for matched dates
if not relevant_rows.empty:
    print("Matched rows:")
    for index, row in relevant_rows.iterrows():
        print(row)
else:
    print("No matching rows found.")

Matched rows:
Name                     Acél Ede
Birthday               1937.01.19
Taj                   007-719-374
Scan Date     2013-06-11 00:00:00
AMD Type                      WET
AMD Type.1                    DRY
Name: 0, dtype: object
